In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import pymc3 as pm
import arviz as az
import os

from math import log10, floor

useratiodata=1 #0 for Scenario B to incorporate ratio data, 0 (Scenario A) otherwise
sigmadeterministic=1 #0 to use plug in estimates of variance of noise variables rather than assign Inverse Gamma prior

information_inputs = [
    {'region': 'CAZ', 'country': 'CAZ Region', 'year': '2021', 'code': 'CAZ'},
    {'region': 'CHA', 'country': 'CHA Region', 'year': '2021', 'code': 'CHA'},
    {'region': 'EUR', 'country': 'EUR Region', 'year': '2021', 'code': 'EUR'},
    {'region': 'IND', 'country': 'India', 'year': '2021', 'code': 'IND'},
    {'region': 'JPN', 'country': 'Japan', 'year': '2021', 'code': 'JPN'},
    {'region': 'LAM', 'country': 'LAM Region', 'year': '2021', 'code': 'LAM'},
    {'region': 'MEA', 'country': 'MEA Region', 'year': '2021', 'code': 'MEA'},
    {'region': 'NEU', 'country': 'NEU Region', 'year': '2021', 'code': 'NEU'},
    {'region': 'OAS', 'country': 'OAS Region', 'year': '2021', 'code': 'OAS'},
    {'region': 'REF', 'country': 'REF Region', 'year': '2021', 'code': 'REF'},
    {'region': 'SSA', 'country': 'SSA Region', 'year': '2021', 'code': 'SSA'},
    {'region': 'USA', 'country': 'United States of America', 'year': '2021', 'code': 'USA'},
]

for information in information_inputs:

    #data information and labelling
    regionname = information['region']
    countryname = information['country']
    countrylabel = information['code'].lower()
    yearlabel = information['year']

    #input data
    dataflows = pd.read_excel("data/input"+"-"+countrylabel+".xlsx", sheet_name='flows-input')
    datastocks = pd.read_excel("data/input"+"-"+countrylabel+".xlsx", sheet_name='changesinstocks-input')
    dataflowspriors = pd.read_excel("data/input"+"-"+countrylabel+".xlsx", sheet_name='flows-prior-input')
    datastockspriors = pd.read_excel("data/input"+"-"+countrylabel+".xlsx", sheet_name='changesinstocks-prior-input')
    dataratios = pd.read_excel("data/input"+"-"+countrylabel+".xlsx", sheet_name='ratios-input')
    
    print(f"BaMFA is started for {countryname}, {yearlabel}, {regionname}")

    #preprocessing
    from preprocessingagg import preprocessing, createdesignmatrix, createratiomatrix, createcompactmatrix, createcompactratiomatrix
    from prior import round_to_poweroften, definepriors
    from model import mfamodel
    from posteriorpredictive import filelabeler, ppplots, ppplotsratiodata, top10hdi

    availabledatafull, dataparentstockneededcols, dataparentflowsneededcols, processnamesdict, allflownumbersmatrix, m, N=preprocessing(datastocks,dataflows)

    #construct the prior

    priormean,covariancevec,truevalues=definepriors(datastockspriors, dataflowspriors, availabledatafull, m, N)

    #construct design matrix

    designmatrix,datavector,availablechildstocksandflows,zerostocksandflows,stockindex,flowindex,CoMindex=createdesignmatrix(availabledatafull, dataparentstockneededcols, dataparentflowsneededcols, m, N)

    #construct matrices for flow ratio data

    ratiovector,ratiomatrixtop,ratiomatrixbottom,availablechildstocksandflows=createratiomatrix(dataratios, m, N, availablechildstocksandflows)

    #set function assures uniqueness of the variables selected
    availablechildstocksandflows=sorted(list(set(availablechildstocksandflows)))

    zerostocksandflows=sorted(list(set(zerostocksandflows)))

    availablechildstocksandflows = [x for x in availablechildstocksandflows if x not in zerostocksandflows]

    availablechildstocks = [i for i in availablechildstocksandflows if i < m]
    availablechildflows = [i for i in availablechildstocksandflows if i >= m]

    availablechildstocksnames=[processnamesdict[str(x)] for x in availablechildstocks]

    #subset of designmatrix for stocks and flows which actually exist in the system/are non zero.
    designmatrixcompact,designmatrixstockscompact,designmatrixflowscompact=createcompactmatrix(designmatrix,availablechildstocksandflows,m)

    ratiomatrixtopstockscompact,ratiomatrixtopflowscompact,ratiomatrixbottomstockscompact,ratiomatrixbottomflowscompact=createcompactratiomatrix(ratiomatrixtop,ratiomatrixbottom,availablechildstocksandflows,m)

    priormeancompact = priormean[availablechildstocksandflows]
    covarianceveccompact = covariancevec[availablechildstocksandflows]
    priorcovariancecompact = np.diag(covarianceveccompact)

    truevaluescompact=truevalues[availablechildstocksandflows]   

    #run the Bayesian model

    trace, model=mfamodel(priormean, covariancevec, designmatrix,ratiomatrixtop, \
                       ratiomatrixbottom, datavector, ratiovector, availablechildstocksandflows, m, \
                       stockindex, flowindex, CoMindex,useratiodata,sigmadeterministic)

    #summary of samples including rhat values for each posterior variable

    az.summary(trace).round(2)

    #generate traceplots for the change in stock variables

    labels=[processnamesdict[str(x)] for x in availablechildstocks]

    import arviz.labels as azl

    directory = "results"+"/"+countrylabel+"/"+"outputgraphstrace" + filelabeler(useratiodata)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    plt.rcParams['font.family'] = 'Arial'

    plt.tight_layout()
    for i in range(0,len(labels)):

        labeller = azl.MapLabeller(var_name_map={"stocks": r"Stock"+":"+labels[i]})
        plt.figure()
        posteriorstocktraceplots=az.plot_trace(trace, var_names="stocks",compact=True,show=True,backend="matplotlib", coords={'stocks_dim_0': [i]},labeller=labeller,legend=True,chain_prop={"color": ['r', 'b']})
        posteriorstocktraceplots[0,0].get_figure().savefig("results"+"/"+countrylabel+"/"+"outputgraphstrace"+filelabeler(useratiodata)+"/"+"Stocktrace"+"_"+labels[i]+filelabeler(useratiodata)+".pdf")

    #generate traceplots for the flow variables

    az.rcParams["plot.max_subplots"] = 400 #this increases the maximum plots displayed, default is 40.
    #There are around 200 posterior flow plots so without this you only plot the first 40. 

    plt.tight_layout()

    directory = "results"+"/"+countrylabel+"/"+"outputgraphstrace" + filelabeler(useratiodata)
    if not os.path.exists(directory):
        os.makedirs(directory)

    for i in range(0, len(availablechildflows)):
        relevantrow=np.where(allflownumbersmatrix[:, 0] == str(availablechildflows[i]))
        relevantrow=relevantrow[0][0]
        flownumberfrom=allflownumbersmatrix[relevantrow, 1]
        flownumberto=allflownumbersmatrix[relevantrow, 2]

        labeller = azl.MapLabeller(var_name_map={"flows": processnamesdict[str(flownumberfrom)]+"_to_"+processnamesdict[str(flownumberto)]})
        plt.figure()
        posteriorflowtraceplots=az.plot_trace(trace, var_names="flows",compact=True,show=True,backend="matplotlib", coords={'flows_dim_0': [i]},labeller=labeller,legend=True,chain_prop={"color": ['r', 'b']})
        posteriorflowtraceplots[0,0].get_figure().savefig("results"+"/"+countrylabel+"/"+"outputgraphstrace"+filelabeler(useratiodata)+"/"+"Flowtrace"+processnamesdict[str(flownumberfrom)]+"_to_"+processnamesdict[str(flownumberto)]+filelabeler(useratiodata)+".pdf")
        plt.close()

    #plot marginal posterior distribution for change in stock variables

    labels=[processnamesdict[str(x)] for x in availablechildstocks]

    directory = "results"+"/"+countrylabel+"/"+"outputgraphs" + filelabeler(useratiodata)
    if not os.path.exists(directory):
        os.makedirs(directory)

    for i in range(0,len(labels)):
        plt.figure() 
        posteriorstockplots=az.plot_posterior(trace, var_names="stocks",show=False,backend="matplotlib",round_to=3, hdi_prob=0.95,  coords={'stocks_dim_0': [i]}, textsize=16,figsize=[9.6, 2.4])
        posteriorstockplots.title.set_text("Stock"+":"+labels[i]) 
        plt.axvline(x=truevalues[availablechildstocks][i], color="red")
        posteriorstockplots.get_figure().savefig("results"+"/"+countrylabel+"/"+"outputgraphs"+filelabeler(useratiodata)+"/"+"Stock"+"_"+labels[i]+filelabeler(useratiodata)+".pdf")

    #plot marginal posterior distribution for flow variables

    az.rcParams["plot.max_subplots"] = 400 #this increases the maximum plots displayed, default is 40.
    #There are around 200 posterior flow plots so without this you only plot the first 40. 

    for i in range(0, len(availablechildflows)):
        relevantrow=np.where(allflownumbersmatrix[:, 0] == str(availablechildflows[i]))
        relevantrow=relevantrow[0][0]
        flownumberfrom=allflownumbersmatrix[relevantrow, 1]
        flownumberto=allflownumbersmatrix[relevantrow, 2]
        
        plt.figure()        
        posteriorflowplots=az.plot_posterior(trace, var_names="flows",show=False,backend="matplotlib", round_to=3, hdi_prob=0.95,coords={'flows_dim_0': [i]}, textsize=16, figsize=[9.6, 2.4])
        plt.axvline(x=truevalues[availablechildflows][i], color="red")
        posteriorflowplots.title.set_text(processnamesdict[str(flownumberfrom)]+" to "+processnamesdict[str(flownumberto)])
        posteriorflowplots.get_figure().savefig("results"+"/"+countrylabel+"/"+"outputgraphs"+filelabeler(useratiodata)+"/"+processnamesdict[str(flownumberfrom)]+"_to_"+processnamesdict[str(flownumberto)]+filelabeler(useratiodata)+".pdf")

    trace.to_netcdf("results"+"/"+countrylabel+"/"+"model"+filelabeler(useratiodata)+"-"+countrylabel+".nc")

    #posterior predictive samples

    directory = "results"+"/"+countrylabel+"/"+"posteriorchecks" + filelabeler(useratiodata)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    print(f"processing posterior indexes for {countryname}, {yearlabel}, {regionname}")

    posterior_pred=pm.sample_posterior_predictive(trace=trace, model=model, random_seed=123456)

    #plot posterior predictive 95% HDI and p values, for flow and change in stock data, and CoM conditions.

    ppplots(posterior_pred,datavector,stockindex,flowindex,CoMindex,useratiodata,countrylabel)

    #plot posterior predictive 95% HDI and p values, for ratio data, scenario B only.

    ppplotsratiodata(posterior_pred,ratiovector,useratiodata,countrylabel)

    #plot top 10 widths of marginal posterior 95% HDI

    ci_95_length=top10hdi(trace,processnamesdict,availablechildstocksandflows,useratiodata,m,countrylabel)

    #output for sankey diagram

    output=az.summary(trace).round(9)
    
    from outputforsankey import output_for_sankey
    
    output_for_sankey(output, availablechildflows, availablechildstocks,allflownumbersmatrix,countrylabel)
    
    print(f"BaMFA is completed for {countryname}, {yearlabel}, {regionname}")